In [1]:
import random
from glob import glob

In [2]:
def getRandomProblem(num_towns, num_clouds, loc_town, pop, loc_cloud, cloud_range):
    locations = [random.randint(1, loc_town) for _ in range(num_towns)]
    populations = [random.randint(1, pop) for _ in range(num_towns)]
    
    clouds = list(range(1, loc_cloud + 1))
    random.shuffle(clouds)
    clouds = clouds[:num_clouds]
    cloud_ranges = [random.randint(0, cloud_range) for _ in range(num_clouds)]
    return locations, populations, clouds, cloud_ranges

## Brute-force solution

In [3]:
def removeOne(populations, locations, clouds, cloud_ranges, skip):
    cloudMap = {}
    
    for i in range(len(clouds)):
        if i == skip:
            continue
            
        c, r = clouds[i], cloud_ranges[i]
        for i in range(max(1, c - r), c + r + 1):
            cloudMap[i] = 1
    
    # print(cloudMap)
    sunnyPop = 0
    for l, p in zip(locations, populations):
        if l not in cloudMap:
            sunnyPop += p
    
    return sunnyPop

def bruteForce(populations, locations, clouds, cloud_ranges):
    sunnyPops = []
    for i in range(len(clouds)):
        sunnyPop = removeOne(populations, locations, clouds, cloud_ranges, i)
        sunnyPops.append(sunnyPop)
        # print(f'skip = {i}, sunny population = {sunnyPop}')
    
    return sunnyPops

## Generate a few random datasets

In [4]:
count = 0
for i in range(20):
    
    print(f'\n{i}')
    locations, populations, clouds, cloud_ranges = getRandomProblem(5, 7, 30, 1000, 30, 10)
    sunnyPops = bruteForce(populations, locations, clouds, cloud_ranges)
    pop_min, pop_max = min(sunnyPops), max(sunnyPops)

    
    if pop_min < pop_max:
        count += 1
        fname = f'data/sample{str(count).zfill(2)}.txt'
        with open(fname, 'w') as handle:
            handle.write(str(len(locations)) + '\n')
            handle.write(' '.join(map(str, locations)) + '\n')
            handle.write(' '.join(map(str, populations)) + '\n')
            handle.write(str(len(clouds)) + '\n')
            handle.write(' '.join(map(str, clouds)) + '\n')
            handle.write(' '.join(map(str, cloud_ranges)) + '\n')
            handle.write(str(pop_max) + '\n')


0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19


In [5]:
def getSunnyIntervals(clouds, a, b, i):
    """
    get sunny intervals with 
    - min location = a
    - max location = b
    - min index in clouds = i
    """
    sunnyIntervals = []
    while a <= b and i < len(clouds) and clouds[i][0] <= b:
        if clouds[i][0] > a:
            sunnyIntervals.append([a, clouds[i][0] - 1])
        a = max(a, clouds[i][1] + 1)
        i += 1
    if b >= a:
        sunnyIntervals.append([a, b])
    return sunnyIntervals

    
def getCount(intervals, locations, pop, start):
    i, j, count = 0, start, 0
    while j < len(locations):
        x = locations[j]
        while i < len(intervals) and intervals[i][1] < x:
            i += 1
        if i == len(intervals):
            break
        if x >= intervals[i][0]:
            count += pop[x]
        j += 1
    return count, j
   
    
def maximumPeople(P, X, Y, R):    
    pop = {}
    for x, p in zip(X, P):
        if x in pop:
            pop[x] += p
        else:
            pop[x] = p
    locations = sorted(list(pop.keys()))
    # print(pop)
    
    clouds = sorted([(max(1, y - r), y + r) for y, r in zip(Y, R)])
    # print(clouds)
    
    l = 0
    maxRelieved, last = 0, 0
    for i, c in enumerate(clouds):
        oci = getSunnyIntervals(clouds, max(last + 1, c[0]), c[1], i + 1)
        mr = 0
        for lower, upper in oci:
            while l < len(locations) and locations[l] <= upper:
                if locations[l] >= lower:
                    mr += pop[locations[l]]
                l += 1
#         print(c)
#         print(oci)
#         print(l)
#         print(locations[l])
        maxRelieved = max(maxRelieved, mr)
        if l == len(locations):
            break
        last = max(last, c[1])
    
    sunnyIntervals = getSunnyIntervals(clouds, 1, locations[-1], 0)
    sunnyPop, _ = getCount(sunnyIntervals, locations, pop, 0)
            
    # print(f'sunny intervals = {sunnyIntervals}')
    # print(f'sunny population = {sunnyPop}')
    
    print(maxRelieved)
    print(sunnyPop)
    return maxRelieved + sunnyPop

In [6]:
def loadProblem(fname):

    with open(fname, 'r') as handle:
        len_X = int(handle.readline().strip())
        
        X = list(map(int, handle.readline().strip().split()))
        P = list(map(int, handle.readline().strip().split()))
        
        len_C = int(handle.readline().strip())
        Y = list(map(int, handle.readline().strip().split()))
        R = list(map(int, handle.readline().strip().split()))        
        
        result = int(handle.readline().strip())
        
    return X, P, Y, R, result

In [8]:
# for fname in glob('data/sample*.txt'):
for fname in glob('data/input07.txt'):    
    
    print(f'\n================= {fname} =================')
    X, P, Y, R, result = loadProblem(fname)
    # print(f'town locations = {X}')
    # print(f'town populations = {P}')
    # print(f'Cloud locations = {Y}')
    # print(f'Cloud ranges = {R}')
    print(f'result = {result}')
    
    myResult = maximumPeople(P, X, Y, R)
    print(f'my result = {myResult}')
    
    # bruteForce(P, X, Y, R)
    print(f'=====================================================\n')


================= data/input07.txt =================
result = 90394853496857
3338972522
90411742692089
my result = 90415081664611



In [155]:
for _ in range(50):
    X, P, Y, R = getRandomProblem(30, 10, 60, 1000, 60, 20)
    result = max(bruteForce(P, X, Y, R))
    myResult = maximumPeople(P, X, Y, R)
    print(result)
    
    if result != myResult:
        print(f'town locations = {X}')
        print(f'town populations = {P}')
        print(f'Cloud locations = {Y}')
        print(f'Cloud ranges = {R}')
        print(f'result = {result}')
        print(f'my result = {myResult}')

5842
709
0
0
0
2099
1661
0
0
1360
2625
729
564
1271
5404
2191
1336
695
547
2185
0
0
2300
4100
1001
3409
0
1356
0
2320
770
3749
0
3931
1466
2567
447
0
1417
0
1106
7139
480
5766
1867
477
1794
0
1078
0
